In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2

In [ ]:
from google.colab.patches import cv2_imshow

def count_circles(image_path):
  """
  Counts circle-like objects in a grayscale image and generates an output
  image with the detected circles visualized.

  Args:
      image_path: Path to the grayscale image file.

  Returns:
      The number of circle-like objects detected in the image.
  """
  # Read the image in grayscale
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  # Apply blur to reduce noise
  blurred_img = cv2.GaussianBlur(img, (5, 5), 0)

  # Apply thresholding to convert the image to binary
  thresh = cv2.threshold(blurred_img, 127, 255, cv2.THRESH_BINARY)[1]

  # Find contours in the thresholded image
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Initialize the count of circles
  count = 0

  # Load the original image for visualization
  output_img = cv2.imread(image_path)

  for cnt in contours:
    # Approximate the contour with a circle
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)

    # Calculate width and height
    w = radius * 2
    h = radius * 2

    # Prevent division by zero
    if h == 0:
      continue

    # Check if contour is circular enough
    aspect_ratio = abs(1 - float(w) / h)
    if aspect_ratio < 0.2:
      # Draw circle on the output image
      cv2.circle(output_img, center, radius, (0, 255, 0), 2)
      count += 1

  # Save the output image
  cv2.imwrite("output_image.jpg", output_img)

  return count

# Example usage
image_path = "/content/drive/MyDrive/Image Processing - Researcher/SEM_contrast/500x-2.jpg"  # Replace with your image path
number_of_circles = count_circles(image_path)

print(f"Number of circle-like objects detected: {number_of_circles}")




Number of circle-like objects detected: 112


In [ ]:
#SEM_contrast

import cv2
import numpy as np


def calculate_gray_area(image_path, lower_gray=50, upper_gray=200):
    """
    Calculates the percentage area occupied by pixels within a specified grayscale range.

    Args:
        image_path: The path to the grayscale image.
        lower_gray: The lower bound of the gray range (default: 50).
        upper_gray: The upper bound of the gray range (default: 200).

    Returns:
        A tuple containing:
            - The percentage area of the image occupied by gray pixels.
            - A binary image where gray pixels are highlighted in white.
            - The original grayscale image.
    """

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image
    mask = cv2.inRange(img, lower_gray, upper_gray)
    total_pixels = img.size
    gray_pixels = np.count_nonzero(mask)
    percentage_gray = (gray_pixels / total_pixels) * 100

    return percentage_gray, mask, img

def visualize_gray_area(img, mask, color=(0, 255, 0)):
    """
    Visualizes the calculated gray area in color.

    Args:
        img: The original grayscale image.
        mask: The binary mask where gray pixels are highlighted.
        color: The color to use for highlighting the gray area (BGR format, default: green).
    """

    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Convert to color
    color_img[mask == 255] = color
    cv2.imwrite("output_gray_area.jpg", color_img)

image_path = "/content/drive/MyDrive/Image Processing - Researcher/SEM_contrast/500x-2.jpg"
percentage_area, mask, img = calculate_gray_area(image_path, lower_gray=80, upper_gray=150)

print(f"Percentage area of gray pixels: {percentage_area:.2f}%")

visualize_gray_area(img, mask, color=(0, 0, 255))  # Visualize in red
#cv2.imshow("Visualized Gray Area", color_img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()



Percentage area of gray pixels: 43.90%


In [5]:
#SEM_color

import cv2
import numpy as np

def calculate_color_area(image_path, lower_color, upper_color):
    """
    Calculates the percentage area occupied by pixels within a specified color range in HSV color space.

    Args:
        image_path: The path to the image.
        lower_color: The lower bound of the color range (HSV format).
        upper_color: The upper bound of the color range (HSV format).

    Returns:
        A tuple containing:
            - The percentage area of the image occupied by the specified color.
            - A binary image where the specified color pixels are highlighted in white.
    """

    img = cv2.imread(image_path)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # Convert to HSV
    mask = cv2.inRange(hsv_img, lower_color, upper_color)
    total_pixels = img.size
    color_pixels = np.count_nonzero(mask)
    percentage_color = (color_pixels / total_pixels) * 100

    return percentage_color, mask

def visualize_color_area(image_path, percentage, mask, color, output_filename):
    """
    Visualizes the calculated color area on the original image and saves it with the percentage.

    Args:
        image_path: The path to the image.
        percentage: The percentage of the specified color.
        mask: The binary mask where color pixels are highlighted.
        color: The color to use for highlighting the color area (BGR format).
        output_filename: The name of the output file to save the image.
    """

    img = cv2.imread(image_path)
    color_img = img.copy()
    color_img[mask == 255] = color

    # Add text showing the percentage
    cv2.putText(color_img, f"{percentage:.2f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imwrite(output_filename, color_img)

image_path = "/content/drive/MyDrive/Image Processing - Researcher/SEM_color/cropped_SEM_color.JPG"

# Red color range
#lower_red = np.array([130, 50, 80])
#upper_red = np.array([180, 255, 255])
lower_red = np.array([150, 80, 100])
upper_red = np.array([180, 255, 255])

# Combine masks to get the entire range of reds
#img = cv2.imread(image_path)
#hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#mask_red1 = cv2.inRange(hsv_img, lower_red1, upper_red1)
#mask_red2 = cv2.inRange(hsv_img, lower_red2, upper_red2)
#mask_red = cv2.bitwise_or(mask_red1, mask_red2)
percentage_red, mask_red = calculate_color_area(image_path, lower_red, upper_red)

# Blue color range
#lower_blue = np.array([120, 50, 50])
#upper_blue = np.array([150, 200, 200])    #this gives 11.45%
#lower_blue = np.array([100, 30, 30])
#upper_blue = np.array([160, 255, 255])    #this gives 19.38%
lower_blue = np.array([100, 30, 0])
upper_blue = np.array([170, 255, 255])     #25.82% but not ok
percentage_blue, mask_blue = calculate_color_area(image_path, lower_blue, upper_blue)


print(f"Percentage area of red pixels: {percentage_red:.2f}%")
print(f"Percentage area of blue pixels: {percentage_blue:.2f}%")

# Visualize
visualize_color_area(image_path, percentage_red, mask_red, (255, 255, 255), "output_red_area.jpg")
visualize_color_area(image_path, percentage_blue, mask_blue, (255, 255, 255), "output_blue_area.jpg")



Percentage area of red pixels: 17.39%
Percentage area of blue pixels: 25.82%


In [2]:
mask_red_or_blue = cv2.bitwise_or(mask_red, mask_blue)\

img = cv2.imread(image_path)
color_img = img.copy()   # Make a copy of the original image

# Set pixels identified by the mask to a specific color (e.g., white)
color_img[mask_red_or_blue == 255] = (255, 255, 255)

output_filename = "output_red_or_blue_area.jpg"
cv2.imwrite(output_filename, color_img)

True